# Scraping: récupération de la page d'accueil du journal Le Soir

Dans ce notebook, nous créons un robot qui va ouvrir la page d'accueil du site du journal [Le Soir](https://www.lesoir.be/) et récupérer le titre de tous les articles du jour et les stocker dans un fichier csv.

## Imports

In [2]:
import os
import re
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Récupération de tous les articles de la page d'accueil



In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

articles = []

root_url = f"https://max.de.wilde.web.ulb.be/camille/"
response = requests.get(root_url, headers=headers,verify=False)
soup = BeautifulSoup(response.text, 'html.parser')
for link in soup.find_all("a"):
    title = link.text.strip()
    url = root_url + link.get('href')
    articles.append([url, title])

c:\Users\sophi\tac\tac_venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'max.de.wilde.web.ulb.be'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [4]:
# Affichage du nombre d'articles récupérés
len(articles)

51

In [5]:
# Affichage des 10 premières entrées
articles[:10]

[['https://max.de.wilde.web.ulb.be/camille/KB_JB230_1892-08-07_01-0003.pdf',
  'KB_JB230_1892-08-07_01-0003.pdf'],
 ['https://max.de.wilde.web.ulb.be/camille/KB_JB427_1920-01-10_01-00004.pdf',
  'KB_JB427_1920-01-10_01-00004.pdf'],
 ['https://max.de.wilde.web.ulb.be/camille/KB_JB555_1836-02-08_01-00002.pdf',
  'KB_JB555_1836-02-08_01-00002.pdf'],
 ['https://max.de.wilde.web.ulb.be/camille/KB_JB638_1860-05-21_01-00002.pdf',
  'KB_JB638_1860-05-21_01-00002.pdf'],
 ['https://max.de.wilde.web.ulb.be/camille/KB_JB773_1918-11-30_01-00002.pdf',
  'KB_JB773_1918-11-30_01-00002.pdf'],
 ['https://max.de.wilde.web.ulb.be/camille/KB_JB838_1887-12-28_01-00003.pdf',
  'KB_JB838_1887-12-28_01-00003.pdf'],
 ['https://max.de.wilde.web.ulb.be/camille/KB_JB230_1903-10-16_01-0002.pdf',
  'KB_JB230_1903-10-16_01-0002.pdf'],
 ['https://max.de.wilde.web.ulb.be/camille/KB_JB427_1933-01-04_01-00003.pdf',
  'KB_JB427_1933-01-04_01-00003.pdf'],
 ['https://max.de.wilde.web.ulb.be/camille/KB_JB555_1899-01-19_01-00

## Création d'un dataframe avec les liens et les titres des articles


In [ ]:
df = pd.DataFrame(articles, columns=['link', 'title'])
df

In [6]:
# Sauvegarde du dataframe dans un fichier csv
df.to_csv(f"../data/rtbf_{time.strftime('%Y%m%d')}.csv", index=False)

NameError: name 'df' is not defined

## Téléchargement d'un article et affichage du texte

In [7]:
# Récupération du premier lien du dataframe
article_url = df.iloc[0][0]

# Ouverture de la page
response = requests.get(article_url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
# Parsing du contenu
title = soup.find("h1").get_text(separator=' ')

paragraphs = []
for paragraph in soup.find_all("p", attrs={"class": None}):
    paragraphs.append(paragraph.get_text(separator=' ').strip())

content = "\n".join(paragraphs)
print(title)
print("==================================")
print(content)

NameError: name 'df' is not defined

### Nettoyage du texte à l'aide d'expressions régulières

In [8]:
# Suppression de tout ce qui se trouve entre parenthèses
clean_content = re.sub("\([^\)]+\)", " ", content)
# Suppression des espaces multiples
clean_content = re.sub("\s+", " ", clean_content)

print(content)
print("==================================")
print(clean_content)


NameError: name 'content' is not defined

### Création d'un fichier avec le contenu de l'article


In [ ]:
with open("../data/rtbf_example.txt", "w") as writer:
    writer.write(f"{title}\n\n{clean_content}")

## Téléchargement automatique des fichiers PDF

Le bloc suivant recherche tous les liens se terminant par .pdf sur la page racine et télécharge chaque fichier dans le dossier `../data/PDF`. Les fichiers déjà présents sont ignorés.


In [ ]:
from urllib.parse import urljoin, urlparse
import os
import requests
from tqdm import tqdm

root = 'https://max.de.wilde.web.ulb.be/camille/'
os.makedirs('../data/PDF', exist_ok=True)

# récupérer la page racine
resp = requests.get(root, headers=headers, verify=False)
soup = BeautifulSoup(resp.text, 'html.parser')

pdf_links = set()
for a in soup.find_all('a', href=True):
    href = a['href']
    if href.lower().endswith('.pdf'):
        full = urljoin(root, href)
        pdf_links.add(full)

print(f'Found {len(pdf_links)} PDF links')

for url in sorted(pdf_links):
    filename = os.path.basename(urlparse(url).path)
    outpath = os.path.join('..', 'data', 'PDF', filename)
    if os.path.exists(outpath):
        print(f'Skipping existing {filename}')
        continue
    try:
        print(f'Downloading {filename}...')
        r = requests.get(url, headers=headers, stream=True, timeout=30)
        r.raise_for_status()
        total = int(r.headers.get('content-length', 0))
        with open(outpath, 'wb') as f:
            if total == 0:
                # no content-length header; write all at once
                f.write(r.content)
            else:
                # stream with progress bar
                with tqdm(total=total, unit='B', unit_scale=True, desc=filename) as pbar:
                    for chunk in r.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)
                            pbar.update(len(chunk))
        print(f'Downloaded {filename}')
    except Exception as e:
        print(f'Failed to download {url}: {e}')

print('Done')

c:\Users\sophi\tac\tac_venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'max.de.wilde.web.ulb.be'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Found 51 PDF links
Failed to download https://max.de.wilde.web.ulb.be/camille/KB_JB230_1892-08-07_01-0003.pdf: HTTPSConnectionPool(host='max.de.wilde.web.ulb.be', port=443): Max retries exceeded with url: /camille/KB_JB230_1892-08-07_01-0003.pdf (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')))
Failed to download https://max.de.wilde.web.ulb.be/camille/KB_JB230_1903-10-16_01-0002.pdf: HTTPSConnectionPool(host='max.de.wilde.web.ulb.be', port=443): Max retries exceeded with url: /camille/KB_JB230_1903-10-16_01-0002.pdf (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')))
Failed to download https://max.de.wilde.web.ulb.be/camille/KB_JB230_1913-07-05_01-0001.pdf: HTTPSConnectionPool(host='max.de.wilde.web.ulb.be', port=443): Max retries exceeded with url: /camille/

## PoC : récupérer des articles, détecter la langue, traduire et sauvegarder

Le bloc suivant utilise les clients API définis dans `tps/tp1/apis` :
- `gnews_client` pour récupérer des articles (requiert `GNEWS_API_KEY`)
- `libretranslate_client` pour détecter et traduire les descriptions
- `arxiv_client` comme secours si pas de clé GNews


In [ ]:
# Imports pour le PoC
import os
import pandas as pd
from tps.tp1.apis.gnews_client import search_news
from tps.tp1.apis.libretranslate_client import detect_language, translate
from tps.tp1.apis.arxiv_client import search_arxiv

os.makedirs(os.path.join('..','data'), exist_ok=True)


In [ ]:
# Fetch 5 articles (GNews) ou fallback arXiv
query = 'Camille'
items = []
try:
    items = search_news(query, lang='fr', max_results=5)
except Exception as e:
    print('GNews failed or no API key; falling back to arXiv:', e)
    items = []
    for a in search_arxiv(query, max_results=5):
        items.append({'title': a.get('title'), 'url': a.get('url'), 'source': 'arXiv', 'publishedAt': a.get('published'), 'description': a.get('summary')})

# Detect & translate
for it in items:
    descr = it.get('description') or ''
    lang = detect_language(descr) if descr else None
    it['detected_lang'] = lang
    it['description_en'] = translate(descr, source=lang or 'auto', target='en') if descr else ''

df = pd.DataFrame(items)
display(df)
out = os.path.join('..','data','news_demo_from_notebook.csv')
df.to_csv(out, index=False)
print('Saved to', out)


## Pour en savoir plus

- Le web scraping avec Python: https://realpython.com/beautiful-soup-web-scraper-python/ 
- Tutoriel sur les expressions régulières: https://www.w3schools.com/python/python_regex.asp 